# Binary logistic regression model

In [1]:
import tensorflow as tf
import numpy as np

/home/pablo/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


### Create placeholders for inputs and labels

In [2]:
# Inputs
x = tf.placeholder(shape=(None,2),dtype=tf.float32, name="inputs")
# Labels
y = tf.placeholder(shape=(None,1),dtype=tf.float32, name="labels")

### Create Variable tensors for weights and bias
This tensors are initialized with random values

In [3]:
w = tf.Variable(tf.random_normal([2,1]), dtype=tf.float32, name="weight")
b = tf.Variable(tf.random_normal([1]), dtype=tf.float32, name="bias")

def linear_layer(x, w, b):
    return tf.matmul(x,w) + b

init = tf.global_variables_initializer()

### Building the model

In [4]:
output = tf.sigmoid(linear_layer(x, w, b))
output

<tf.Tensor 'Sigmoid:0' shape=(?, 1) dtype=float32>

### Cost and Optimizer

In [14]:
l = 1
with tf.name_scope("Cost_and_Optimizer"):
    cost = tf.reduce_sum(tf.nn.sigmoid_cross_entropy_with_logits(labels=y,logits=output,name="cost"))
    optimizer = tf.train.GradientDescentOptimizer(l)
    train = optimizer.minimize(cost)

In [15]:
x_train = []
y_train = []

import csv

with open("./data/example_1.csv", 'r', newline='') as csvfile:
    reader = csv.DictReader(csvfile)
    for row in reader:
        x_train.append([float(row['x1']),float(row['x2'])])
        y_train.append([float(row['y'])])
        
x_train = np.array(x_train)
y_train = np.array(y_train)

In [18]:
tf.summary.scalar("Cost", cost)
tf.summary.histogram("weights", w)
tf.summary.histogram("biases", b)
epoch = 1000

with tf.Session() as sess:
    merged_summary = tf.summary.merge_all()
    writer = tf.summary.FileWriter("./summaries/2")
    writer.add_graph(sess.graph)
    sess.run(init)

    ## Batch_size = 1
#     for i in range(epoch):
#         for j in range(len(x_train)):
#             sess.run(train, feed_dict={x: [x_train[j]], y: [y_train[j]]})
        ####
    
## Batch_size = m
    for i in range(epoch):
        sess.run(train, feed_dict={x: x_train, y: y_train})

        if i%100 == 0:
            partial_cost = sess.run(cost, {x: x_train, y: y_train})
            outputs = sess.run(output, {x: x_train, y: y_train})
            predictions = outputs > 0.5
           
            accuracy = np.sum(y_train == predictions) / y_train.shape[0]
           
            print("Epoch N°: {} - Cost = {} - Accuracy = {}%".format(i, partial_cost, accuracy * 100))
           
            s = sess.run(merged_summary, {x: x_train, y: y_train})
            writer.add_summary(s, i)
    
        
    final_w, final_b, final_cost = sess.run([w, b, cost], {x: x_train, y: y_train})
    

Epoch N°: 0 - Cost = 2.910815477371216 - Accuracy = 50.0%
Epoch N°: 100 - Cost = 2.069517135620117 - Accuracy = 100.0%
Epoch N°: 200 - Cost = 2.0398621559143066 - Accuracy = 100.0%
Epoch N°: 300 - Cost = 2.030484437942505 - Accuracy = 100.0%
Epoch N°: 400 - Cost = 2.025916337966919 - Accuracy = 100.0%
Epoch N°: 500 - Cost = 2.0232186317443848 - Accuracy = 100.0%
Epoch N°: 600 - Cost = 2.021439552307129 - Accuracy = 100.0%
Epoch N°: 700 - Cost = 2.020179033279419 - Accuracy = 100.0%
Epoch N°: 800 - Cost = 2.0192391872406006 - Accuracy = 100.0%
Epoch N°: 900 - Cost = 2.0185117721557617 - Accuracy = 100.0%


### Final model

In [ ]:
print("Final function: Y = %s * X + %s"%(final_w, final_b))
print("Final cost: %s"%final_cost)

### TensorBoard - Graph visualization
![title](graph.png)